In [ ]:
!pip install pyspark

In [3]:
# importar bibliotecas
import pyspark
from pyspark.sql import SparkSession

#Criação de uma sessão Spark(caso ela não exista) ou recuperação de uma 
#sessão existente com o mesmo nome
spark = SparkSession.builder.appName("Word Count").master('local[*]').getOrCreate()

# criação do contexto

sc = spark.sparkContext

rdd = sc.textFile("RealEstate.csv")
rdd.take(4)

# Ignorar o Cabeçalho

rdd = rdd.filter(lambda x:x.split(",")[0]!="MLS")
rdd.take(4)

['132842,Arroyo Grande,795000.00,3,3,2371,335.30,Short Sale',
 '134364,Paso Robles,399000.00,4,3,2818,141.59,Short Sale',
 '135141,Paso Robles,545000.00,4,3,3032,179.75,Short Sale',
 '135712,Morro Bay,909000.00,4,4,3540,256.78,Short Sale']

In [11]:
# Transformar o RDD em PairRDD com (chave, (valor1, valor2)) -> (nro_rooms, (price, 1))
pairRDD = rdd.map(lambda x:(int(x.split(",")[3]), (float(x.split(",")[2]), 1)))
pairRDD.take(4)

[(3, (795000.0, 1)),
 (4, (399000.0, 1)),
 (4, (545000.0, 1)),
 (4, (909000.0, 1))]

In [12]:
# Obter a soma dos preços e a quantidade de casas por chave
pairRDD_sum = pairRDD.reduceByKey(lambda x,y: (x[0]+y[0], x[1]+y[1]))
pairRDD_sum.take(4)

[(4, (85575190.0, 177)),
 (2, (32761834.0, 123)),
 (0, (586900.0, 2)),
 (6, (2412900.0, 4))]

In [14]:
# Calcular a média (soma dos preços / qtde de casas)
pairRDD_average = pairRDD_sum.mapValues(lambda x: x[0] / x[1])
pairRDD_average.collect()

[(4, 483475.6497175141),
 (2, 266356.3739837398),
 (0, 293450.0),
 (6, 603225.0),
 (10, 699000.0),
 (3, 359062.20649651974),
 (7, 325000.0),
 (1, 169981.81818181818),
 (5, 657858.0645161291)]

In [15]:
# Ordenar a saída de maneira crescente de acordo com o nro_room
pairRDD_average_sorted = pairRDD_average.sortByKey(ascending=True)
pairRDD_average_sorted.collect()

[(0, 293450.0),
 (1, 169981.81818181818),
 (2, 266356.3739837398),
 (3, 359062.20649651974),
 (4, 483475.6497175141),
 (5, 657858.0645161291),
 (6, 603225.0),
 (7, 325000.0),
 (10, 699000.0)]